In [1]:
import os

In [2]:
%pwd

'F:\\FAHIM\\Data Science\\y_ml_project\\p2\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'F:\\FAHIM\\Data Science\\y_ml_project\\p2'

In [5]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
# configuration package
from src.chicken_deases_classification.constant import *
from src.chicken_deases_classification.utils.common import read_yaml,create_directories

In [35]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath =PARAMS_FILE_PATH):
        self.config =read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_prepare_callbacks_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [30]:
# components
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [39]:
class PrepareCallback:
    def __init__(self,config :PrepareCallbackConfig):
        self.config = config
        
    @property
    def _create_tb_callback(self):
        timestamp =time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_log_at_{timestamp}"
                                          )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            
            filepath=str(self.config.checkpoint_model_filepath),  
            save_best_only=True
        )
    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callback
        ]

In [40]:
# pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callback()
except Exception as e:
    raise e

[2024-07-12 23:41:22,764: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-12 23:41:22,764: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-12 23:41:22,764: INFO: common: created directory at: artifacts]
[2024-07-12 23:41:22,780: INFO: common: created directory at: artifacts\prepare_callback\callbacks_dir]
[2024-07-12 23:41:22,780: INFO: common: created directory at: artifacts\prepare_callback\tensorboard_log_dir]
